In [25]:
import pandas as pd
import torch
import torch.nn as nn
import torch_explain as te
from torch.nn.functional import one_hot
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

testing_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=32, shuffle=True)

In [6]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 1, 28, 28])
Shape of y: torch.Size([32]) torch.int64


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Device: {device}")

Device: cpu


In [44]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            te.nn.EntropyLinear(28*28, 10, n_classes=10),
            nn.LeakyReLU(),
            nn.Linear(10, 4),
            nn.LeakyReLU(),
            nn.Linear(4, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits

In [45]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): EntropyLinear(
      in_features=784, out_features=10, n_classes=10
      (conceptizator): Conceptizator(activation=identity_bool, threshold=0.5)
    )
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=10, out_features=4, bias=True)
    (3): LeakyReLU(negative_slope=0.01)
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [46]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [47]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [48]:
train(train_dataloader, model, loss_fn, optimizer)

RuntimeError: Expected target size [32, 1], got [32]

In [35]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.898116  [    0/60000]
loss: 1.882080  [ 3200/60000]
loss: 1.870308  [ 6400/60000]
loss: 1.786342  [ 9600/60000]
loss: 1.668773  [12800/60000]
loss: 1.575761  [16000/60000]
loss: 1.585548  [19200/60000]
loss: 1.694227  [22400/60000]
loss: 1.756423  [25600/60000]
loss: 1.452594  [28800/60000]
loss: 1.605987  [32000/60000]
loss: 1.584103  [35200/60000]
loss: 1.532045  [38400/60000]
loss: 1.484498  [41600/60000]
loss: 1.576067  [44800/60000]
loss: 1.365154  [48000/60000]
loss: 1.259859  [51200/60000]
loss: 1.442093  [54400/60000]
loss: 1.394375  [57600/60000]
Test Error: 
 Accuracy: 74.7%, Avg loss: 1.274498 

Epoch 2
-------------------------------
loss: 1.279490  [    0/60000]
loss: 1.184880  [ 3200/60000]
loss: 1.308331  [ 6400/60000]
loss: 1.177508  [ 9600/60000]
loss: 1.042246  [12800/60000]
loss: 1.160855  [16000/60000]
loss: 1.139351  [19200/60000]
loss: 0.844919  [22400/60000]
loss: 0.995309  [25600/60000]
loss: 0.889284  [28800/60000

In [37]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth
